In [ ]:
import geopandas as gpd
from shapely.geometry import Point
from objectnat import simulate_noise

# Define the starting point of the noise source (in this case, a single point)
start_p = gpd.GeoDataFrame(geometry=[Point(347262.80, 6648132.41)], crs=32636)

# Read in the obstacles (buildings) data and convert it to the desired coordinate reference system (CRS)
obstacles = gpd.read_parquet('examples_data/buildings.parquet').to_crs(epsg=32636)

# Read in the trees data and convert it to the desired CRS
trees = gpd.read_parquet('examples_data/trees.parquet').to_crs(epsg=32636)

# Optionally, trees can be set to None if you want to omit the effect of vegetation in the simulation
# trees = None

# Simulate the noise propagation using the `simulate_noise` function
# Parameters:
# - source_points: The GeoDataFrame containing the source of the noise.
# - obstacles: GeoDataFrame representing obstacles like buildings.
# - source_noise_db: The initial noise level of the source (95 dB in this case).
# - geometric_mean_freq_hz: The frequency of the sound (2000 Hz).
# - standart_absorb_ratio: Default absorption ratio for obstacles (0.05).
# - trees: GeoDataFrame of trees (can be None).
# - tree_resolution: The resolution for simulating the effect of trees (set to 4).
# - air_temperature: The temperature of the air in °C (set to 20).
# - target_noise_db: The target noise level to simulate (40 dB).
# - db_sim_step: The step size for the simulation in dB (1 dB).
# - reflection_n: The maximum number of reflections to simulate (4).
# - dead_area_r: The radius for dead areas (no reflection zones, set to 5 meters).
noise = simulate_noise(source_points=start_p,
                       obstacles=obstacles,
                       source_noise_db=95,
                       geometric_mean_freq_hz=2000,
                       standart_absorb_ratio=0.05,
                       trees=trees,
                       tree_resolution=4,
                       air_temperature=20,
                       target_noise_db=40,
                       db_sim_step=1,
                       reflection_n=4,
                       dead_area_r=5,
                       )

In [ ]:
# Visualize the simulation results using the `explore` method of GeoDataFrame to display the noise levels
# The 'plasma' colormap is used, with a minimum noise level of 40 dB
m1 = noise.explore(column='noise_level', cmap='plasma', vmin=40)
m1  # Display the map

### Section for GIF creation

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import numpy as np
from matplotlib import cm


def create_noise_animation(gdf_noise, gdf_obstacles, start_p, buffer_p, gdf_trees=None,
                           output_file="noise_animation.gif"):
    if gdf_trees is None:
        gdf_trees = gpd.GeoDataFrame()

    bounds = start_p.unary_union.buffer(buffer_p).bounds
    minx, miny, maxx, maxy = bounds
    vmin = gdf_noise['noise_level'].min()
    vmax = gdf_noise['noise_level'].max()
    cmap = cm.plasma

    fig, ax = plt.subplots(figsize=(10, 10))
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

    def update_frame(frame):
        ax.clear()
        ax.set_xlim(minx, maxx)
        ax.set_ylim(miny, maxy)

        gdf_trees.plot(ax=ax, edgecolor='green', facecolor='none', linewidth=3)
        gdf_obstacles.plot(ax=ax, facecolor='gray')

        gdf_noise[gdf_noise['noise_level'] > frame].plot(ax=ax, column='noise_level', cmap=cmap, alpha=0.8, vmin=vmin,
                                                         vmax=vmax)
        gdf_noise[gdf_noise['noise_level'] == frame].plot(ax=ax, column='noise_level', cmap=cmap, alpha=1, vmin=vmin,
                                                          vmax=vmax)
        gdf_noise[gdf_noise['noise_level'] == frame - 1].plot(ax=ax, column='noise_level', cmap=cmap, alpha=0.5,
                                                              vmin=vmin, vmax=vmax)
        gdf_noise[gdf_noise['noise_level'] < frame - 1].plot(ax=ax, column='noise_level', cmap=cmap, alpha=0.3,
                                                             vmin=vmin, vmax=vmax)

        ax.set_title(f"Noise Level: {frame} dB", fontsize=20)
        ax.set_axis_off()

    frames = np.arange(gdf_noise['noise_level'].max(), gdf_noise['noise_level'].min() - 1, -1)
    ani = animation.FuncAnimation(fig, update_frame, frames=frames, repeat=False)
    ani.save(output_file, writer='imagemagick', fps=15)

    plt.close()

# Call the function to create the noise animation, using the noise, obstacles, and trees data
# Fill in the buffer_p parameter close to the value specified in the logs when running the simulation.
create_noise_animation(noise, obstacles, start_p, 350, trees)
